In [15]:
import tqdm
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertTokenizerFast
from torch.optim import AdamW as TorchAdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import string
import emoji
import re


import spacy
from bs4 import BeautifulSoup
import gensim
from gensim.models import Word2Vec
import numpy as np
import string
import unidecode
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
import random


def reset_numpy_seed(seed_value=42):
  try:
    # Set NumPy random seed
    import numpy as np
    np.random.seed(seed_value)
    print(f'NumPy random seed set with value: {seed_value}')
  except Exception as e:
    print(f'NumPy random seed was not set: {e}')
  return


def reset_tensorflow_seed(seed_value=42):
  try:
    # Set TensorFlow random seed
    import tensorflow as tf
    success = False
    # Here we have 2 different ways to set the seed
    # depending on the version of TensorFlow
    try:
      tf.random.set_seed(seed_value)
      success = True
    except Exception as e:
      pass
    try:
      tf.set_random_seed(seed_value)
      success = True
    except Exception as e:
      pass
    if success:
      print(f'TensorFlow random seed set with value: {seed_value}')
    else:
      print(f'TensorFlow random seed was not set')
  except Exception as e:
    print(f'TensorFlow random seed was not set: {e}')
  return


def reset_torch_seed(seed_value=42):
  try:
    # Set PyTorch random seed
    import torch
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
      torch.cuda.manual_seed(seed_value)
      torch.cuda.manual_seed_all(seed_value)  # if you are using multiple GPUs
    print(f'PyTorch random seed set with value: {seed_value}')
  except Exception as e:
    print(f'PyTorch random seed was not set: {e}')
  return


def set_random_seeds(seed_value=42):
  # Set Python random seed
  random.seed(seed_value)
  reset_numpy_seed(seed_value)
  reset_tensorflow_seed(seed_value)
  reset_torch_seed(seed_value)
  return


if __name__ == '__main__':
  # Set the desired seed value
  seed = 42

  # Set random seeds
  set_random_seeds(seed)


NumPy random seed set with value: 42
TensorFlow random seed set with value: 42
PyTorch random seed set with value: 42


In [17]:
df = pd.read_csv('../Nitro_NLP_Satire/train.csv')
df_test = pd.read_csv('../Nitro_NLP_Satire/test.csv')

df.head()

,id,title,content,class
0,0,PSD în alertă,Prăbușirea PSD de la altitudinea sigură a celo...,True
1,1,"În amintirea Vioricăi, milioane de români beau...",Moțiunea de cenzură care a doborât guvernul Dă...,True
2,2,"Dramă! Când credea că nu se poate mai rău, un ...",Credeai că ai ajuns la fundul sacului? Înseamn...,True
3,3,"Spania - România, 5-0. „Tricolorii”, îngenunch...","Echipa națională a României a fost umilită, lu...",False
4,4,"Campanie electorală, veselie generală","Toate cresc în campania electorală, cît n-au c...",True


In [18]:
df_test

,id,title,content
0,0,"Financial Times: SUA, Japonia, Australia și In...","SUA, Japonia, Australia și India vor dezvălui ..."
1,1,Guvernul poate muta Ziua Culturii Române de pe...,Guvernul ia în considerare să mute Ziua Cultur...
2,2,Ministrul ucrainean al Apărării susține că au ...,Aleksei Reznikov a afirmat în cadrul unui mesa...
3,3,"Amantul unei angajate ANAF s-a sinucis, nemais...",Poliția anchetează cazul unei morți suspecte a...
4,4,Boloş: Nu este vorba de o taxă nouă pe concedi...,"„Ca ministru al Finanţelor, este important să ..."
...,...,...,...
36664,36664,Cei 77 de pompieri români care au ajutat la st...,"Timp de o săptămână, 77 de pompieri români au ..."
36665,36665,Bill Gates anunță că s-a vaccinat împotriva co...,Fondatorul Microsoft a anunțat chiar el pe Twi...
36666,36666,Atelierul auto al Regelui Mihai este deschis p...,"Potrivit Casei Regale, Atelierul Auto a fost i..."
36667,36667,Bibliotecarii cer salarii mai mari. Protest în...,NaN


In [19]:
sample_sub = pd.read_csv("../Nitro_NLP_Satire/sample_test_solution.csv")

len(sample_sub)

36669

In [20]:
# Definesc niste constante

TEXT_COL = "text"
LABEL_COL = "class"
ID_COL = "id"
device = "cuda"

In [21]:

df[TEXT_COL] = df["title"].astype(str) + ". " + df["content"].astype(str)
df_test[TEXT_COL] = df_test["title"].astype(str) + ". " + df_test["content"].astype(str)

df[TEXT_COL] = df[TEXT_COL].replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")
df_test[TEXT_COL] = df_test[TEXT_COL].replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

# df[LABEL_COL] = df["class"]
# df_test[LABEL_COL] = df_test["class"]

test_ids = df_test[ID_COL]

# Definesc niste coloane constante pentru simplitatea codului

df = df[[LABEL_COL, TEXT_COL]]
df_test[TEXT_COL] = df_test[TEXT_COL]

display(df.head())
display(df_test.head())

,class,text
0,True,PSD în alertă. Prăbușirea PSD de la altitudine...
1,True,"În amintirea Vioricăi, milioane de români beau..."
2,True,"Dramă! Când credea că nu se poate mai rău, un ..."
3,False,"Spania - România, 5-0. „Tricolorii”, îngenunch..."
4,True,"Campanie electorală, veselie generală. Toate c..."


,id,title,content,text
0,0,"Financial Times: SUA, Japonia, Australia și In...","SUA, Japonia, Australia și India vor dezvălui ...","Financial Times: SUA, Japonia, Australia și In..."
1,1,Guvernul poate muta Ziua Culturii Române de pe...,Guvernul ia în considerare să mute Ziua Cultur...,Guvernul poate muta Ziua Culturii Române de pe...
2,2,Ministrul ucrainean al Apărării susține că au ...,Aleksei Reznikov a afirmat în cadrul unui mesa...,Ministrul ucrainean al Apărării susține că au ...
3,3,"Amantul unei angajate ANAF s-a sinucis, nemais...",Poliția anchetează cazul unei morți suspecte a...,"Amantul unei angajate ANAF s-a sinucis, nemais..."
4,4,Boloş: Nu este vorba de o taxă nouă pe concedi...,"„Ca ministru al Finanţelor, este important să ...",Boloş: Nu este vorba de o taxă nouă pe concedi...


In [22]:
STOPWORDS = nltk.corpus.stopwords.words("romanian") # Download stopwords

In [23]:
'''
Aici obtim metafeatures.

Cateva optiuni pe care le-am implementat sunt:

word_count
unique_word_count
url_count
mean_word_length
char_count
punctuation_count

Dupa ce numar aceste variablile (aprox 5 min) salvez csv-urile in file-uri si le incarc dupa.
'''


df['word_count'] = df[TEXT_COL].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test[TEXT_COL].apply(lambda x: len(str(x).split()))

df['unique_word_count'] = df[TEXT_COL].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test[TEXT_COL].apply(lambda x: len(set(str(x).split())))

df['stop_word_count'] = df[TEXT_COL].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
df_test['stop_word_count'] = df_test[TEXT_COL].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

df['url_count'] = df[TEXT_COL].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test[TEXT_COL].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

df['mean_word_length'] = df[TEXT_COL].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test[TEXT_COL].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

df['char_count'] = df[TEXT_COL].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test[TEXT_COL].apply(lambda x: len(str(x)))

df['punctuation_count'] = df[TEXT_COL].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test[TEXT_COL].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

df.to_csv("processed_train.csv", index=False)
df_test.to_csv("processed_test.csv", index=False)

In [24]:
len(df_test)

36669

In [25]:
df = pd.read_csv("processed_train.csv")
df_test = pd.read_csv("processed_test.csv")


In [26]:
'''
In partea asta de cod fac preprocesarea si curatarea de text. 
In functiile astea am cateva functii pentru eliminarea sectiuniilor neimportante de text.
'''

def strip_html(text_list):
    return [BeautifulSoup(text, "html.parser").get_text() for text in text_list]

def remove_between_square_brackets(text_list):
    return [re.sub(r'\[[^]]*\]', '', text) for text in text_list]

def remove_stopwords(text_list, stopwords=STOPWORDS):
    cleaned_texts = []
    for text in text_list:
        final_text = [
            unidecode.unidecode(word.strip().lower())
            for word in text.split()
            if word.strip().lower() not in stopwords and word.strip().isalpha()
        ]
        cleaned_texts.append(" ".join(final_text))
    return cleaned_texts

def remove_punctuation(text_list):
    translator = str.maketrans('', '', string.punctuation)
    return [text.translate(translator) for text in text_list]

def remove_punctuation(text_list):
    translator = str.maketrans('', '', string.punctuation)
    return [text.translate(translator) for text in text_list]

def remove_non_ascii(text_list):
    return [''.join(char for char in text if ord(char) < 128) for text in text_list]

def decode_emojis(text_list):
    return [emoji.demojize(text) for text in text_list]

def process_text(text):
    text = strip_html(text)
    print("Done!")
    text = remove_between_square_brackets(text)
    print("Done!")
    text = remove_stopwords(text)
    print("Done!")
    text = remove_punctuation(text)
    print("Done!")
    text = remove_non_ascii(text)
    print("Done!")
    text = decode_emojis(text)
    print("Done!")
    return text

df['text'] = process_text(df["text"].to_list())
df_test["text"] = process_text(df_test["text"].to_list())

df.to_csv("processed_with_extra_remove_train.csv", index=False)
df_test.to_csv("processed_with_extra_remove_test.csv", index=False)


Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


In [ ]:

# Incarc date deja procesate
df = pd.read_csv("processed_with_extra_remove_train.csv")
df_test = pd.read_csv("processed_with_extra_remove_test.csv")

# Pt debug
df = df.head(10000)
df_test = df_test.head(10)

# Definesc constante
TEXT_COL = "text"
LABEL_COL = "class"
ID_COL = "id"
device = "cuda"
cols_drop = ["url_count"]

# Drop o col useless
df.drop(columns=cols_drop, inplace=True)
df_test.drop(columns=cols_drop, inplace=True)


In [ ]:
def drop_outliers(df, th: dict):
    '''
    Functie pentru a elemina toate valorile care sar un threshold setat
    '''
    METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'mean_word_length',
                'char_count', 'punctuation_count']
    
    for i, meta in enumerate(METAFEATURES):
        df = df[(df[meta] > th[meta][0]) & (df[meta] < th[meta][1])]
    return df

df = drop_outliers(
    df, 
    {
        # Valori determinate folosind ploturile de mai jos
        "word_count": (0, 2500),
        "unique_word_count": (0, 1000),
        "stop_word_count": (0, 600),
        "mean_word_length": (0, 10),
        "char_count": (0, 20000),
        "punctuation_count": (0, 400)
    }
)


In [ ]:
METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'mean_word_length',
                'char_count', 'punctuation_count']
DISASTER_TWEETS = df[LABEL_COL] == 1

fig, axes = plt.subplots(ncols=2, nrows=len(METAFEATURES), figsize=(20, 50), dpi=100)

# Plotez distributia fiecarui feature. Un plot pentru date de test vs. train si unul pentru date True vs. Flase pe datele de train

for i, feature in enumerate(METAFEATURES):
    sns.distplot(df.loc[~DISASTER_TWEETS][feature], label='False', ax=axes[i][0], color='green')
    sns.distplot(df.loc[DISASTER_TWEETS][feature], label='True', ax=axes[i][0], color='red')

    sns.distplot(df[feature], label='Training', ax=axes[i][1])
    sns.distplot(df_test[feature], label='Test', ax=axes[i][1])
    
    for j in range(2):
        axes[i][j].set_xlabel('')
        axes[i][j].tick_params(axis='x', labelsize=12)
        axes[i][j].tick_params(axis='y', labelsize=12)
        axes[i][j].legend()
    
    axes[i][0].set_title(f'{feature} Target Distribution in Training Set', fontsize=13)
    axes[i][1].set_title(f'{feature} Training & Test Set Distribution', fontsize=13)

plt.show()


In [ ]:
df["text"] = df["text"].astype(str)

# Fac textele si labelurile ca liste

texts = df["text"].to_list()
labels = df["class"].to_list()


In [ ]:
# Encode labelurile sub forma one-hot

labels_one_hot = []

for l in labels: 
    if(l == False): labels_one_hot.append([1, 0])
    else: labels_one_hot.append([0, 1])

labels = labels_one_hot

In [ ]:
# Impart datasetul in 3 partie -> train, test, val. Fiecare are 

lim1 = len(texts) // 1.5
lim2 = lim1 + len(texts) // 5;

train_texts, train_labels = texts[:lim1], labels[:lim1]
test_texts, test_labels = texts[lim1 + 1:lim2], labels[lim1 + 1:lim2]
val_texts, val_labels = texts[lim2 + 1:], labels[lim2 + 1:]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dumitrescustefan/bert-base-romanian-cased-v1')
model = BertForSequenceClassification.from_pretrained('dumitrescustefan/bert-base-romanian-cased-v1', num_labels=2).to(device)

In [ ]:
METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'mean_word_length',
                'char_count', 'punctuation_count']

metafeatures = df[METAFEATURES].values

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [ ]:
'''
In partea asta dau pop la cate valori am nevoie din fiecare embeding.
Dupa append valorile obtinute in metafeatures.
Fac procedeul de sus pentru fiecare dataset -> train, test, val
'''

for i in range(len(train_encodings["input_ids"])):
    train_encodings["input_ids"][i] = train_encodings["input_ids"][i][:len(train_encodings["input_ids"][i]) - len(METAFEATURES)]
    train_encodings["attention_mask"][i] = train_encodings["attention_mask"][i][:len(train_encodings["attention_mask"][i]) - len(METAFEATURES)]

    for j in range(len(METAFEATURES)):
        train_encodings["input_ids"][i].append(int(metafeatures[i][j]))
        train_encodings["attention_mask"][i].append(1)



for i in range(len(test_encodings["input_ids"])):
    test_encodings["input_ids"][i] = test_encodings["input_ids"][i][:len(test_encodings["input_ids"][i]) - len(METAFEATURES)]
    test_encodings["attention_mask"][i] = test_encodings["attention_mask"][i][:len(test_encodings["attention_mask"][i]) - len(METAFEATURES)]

    for j in range(len(METAFEATURES)):
        test_encodings["input_ids"][i].append(int(metafeatures[lim1 + i][j]))
        test_encodings["attention_mask"][i].append(1)



for i in range(len(val_encodings["input_ids"])):
    val_encodings["input_ids"][i] = val_encodings["input_ids"][i][:len(val_encodings["input_ids"][i]) - len(METAFEATURES)]
    val_encodings["attention_mask"][i] = val_encodings["attention_mask"][i][:len(val_encodings["attention_mask"][i]) - len(METAFEATURES)]

    for j in range(len(METAFEATURES)):
        val_encodings["input_ids"][i].append(int(metafeatures[lim2 + i][j]))
        val_encodings["attention_mask"][i].append(1)


In [ ]:
# Fac un dataset de tensori pentru fiecare set de date.

train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']),
                                               torch.tensor(train_labels))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']),
                                             torch.tensor(val_encodings['attention_mask']),
                                             torch.tensor(val_labels))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                              torch.tensor(test_encodings['attention_mask']),
                                              torch.tensor(test_labels))

In [ ]:
batch_size = 30

# Dataloaderul imaprte datele in segmente de batch_size lungime.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Setez ca parametrii tranable doar cei de classifier, nu cei de bert.

for param in model.bert.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:

optimizer = TorchAdamW(model.classifier.parameters(), lr=1e-5)
model.train()

for epoch in range(1):
    for batch in tqdm.tqdm(train_loader):
        input_ids, attention_mask, labels = batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        labels = torch.argmax(labels, dim=1)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)


        loss = outputs.loss

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), 'BERT_{i}.pth'.format(i = epoch))

In [ ]:
model.eval()


val_predictions = []
val_true_labels = []

with torch.no_grad():
    for batch in tqdm.tqdm(val_loader):
       input_ids, attention_mask, labels = batch
       input_ids = input_ids.to(device)
       attention_mask = attention_mask.to(device)
       labels = labels.to(device)

       labels = torch.argmax(labels, dim=1)
       logits = model(input_ids, attention_mask=attention_mask).logits
       val_predictions.extend(torch.argmax(logits, dim=1).tolist())
       val_true_labels.extend(labels.tolist())
       
val_accuracy = accuracy_score(val_true_labels, val_predictions)

print("Validation Accuracy:", val_accuracy)

In [ ]:

test_predictions = []
test_true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        logits = model(input_ids, attention_mask=attention_mask).logits
        test_predictions.extend(torch.argmax(logits, dim=1).tolist())
        test_true_labels.extend(labels.tolist())

test_accuracy = accuracy_score(test_true_labels, test_predictions)

print("Test Accuracy:", test_accuracy)

In [ ]:
torch.save(model.state_dict(), 'BERT.pth')